In [1]:
import cv2
import numpy as np


In [2]:
pip install requests


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import requests

def download_file(url, filename):
    """
    Helper function to download a file from a URL.
    """
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                f.write(chunk)

weights_url = 'https://pjreddie.com/media/files/yolov3.weights'
weights_filename = 'yolov3.weights'

download_file(weights_url, weights_filename)
print("Download completed!")


In [7]:
import requests

def download_file(url, filename):
    """
    Helper function to download a file from a URL.
    """
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                f.write(chunk)

# URL for the coco.names file
url = "https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names"
filename = "coco.names"

download_file(url, filename)
print("Download of coco.names completed!")


Download of coco.names completed!


In [2]:
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")

In [3]:
classes = []
with open('coco.names', 'r') as f:
    classes = [line.strip() for line in f.readlines()]

# Set the input size for the YOLOv3 model
input_width = 416
input_height = 416


net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

video_path = "road_trafifc.mp4"
cap = cv2.VideoCapture(video_path)


while True:
    ret, frame = cap.read()
    if not ret:
        break

   
    if frame is None:
        print("Error: Empty frame")
        continue

   
    blob = cv2.dnn.blobFromImage(frame, 1/255, (input_width, input_height), (0, 0, 0), True, crop=False)

   
    net.setInput(blob)

   
    detections = net.forward()

  
    for detection in detections:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.4:
           
            box = detection[0:4] * nqp.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])
            (x, y, w, h) = box.astype("int")

           
            indices = cv2.dnn.NMSBoxes([(x, y, w, h)], [confidence], 0.5, 0.4)

            
            if len(indices) > 0:
                for i in indices.flatten():
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                    text = f"{classes[class_id]}: {confidence:.2f}"
                    cv2.putText(frame, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)



    cv2.imshow("frame",frame)

    
    if cv2.waitKey(1) == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()